<h1>Newton-Raphson<h1> 
<p>A method for a system of n nonlinear equations with n unknowns.</p>

In [ ]:
import torch as th  # Pytorch library
import matplotlib.pyplot as plt

In [ ]:
class NewtonRaphson:

  def __init__(self, x, tol, functions, maxIterations):
    self.x = x  # Initial x vector
    self.tolerance = tol  # Tolerance
    self.functions = functions  
    self.maxIterations = maxIterations  # Max iterations 

  # Calculate the jacobian matrix
  # The Jacobian method in pytorch calculates n unknown variables for one function so
  # for n functions we loop through n times and store the results in a list.
  def jacobianMatrix(self):
    jacobian = []
    for i in range(len(self.functions)):
      jacobian.append(list(map(lambda x: x.item(), list(th.autograd.functional.jacobian(self.functions[i], self.x)))))
    return th.tensor(jacobian, dtype=th.float64)
  
  # Calculate inverse jacobian
  def inverseJacobian(self):
    return th.inverse(self.jacobianMatrix())

  # Calculate the functions by replacing the unknown variables with their values
  def functionCalculation(self):
    functionValues = []
    for function in self.functions:
      functionValues.append(function(self.x))
    return th.tensor([functionValues], dtype=th.float64)

  # Check if the local error for all the unknown variables is greater than the tolerance
  def islocalErrorGreaterThanTolerance(self, counter, local_variable_errors):
    errorsGreaterThanTolerance = 0
    for i in range(len(self.x)):
      if local_variable_errors[counter][i] >= self.tolerance:
        errorsGreaterThanTolerance += 1
    
    return True if errorsGreaterThanTolerance == len(self.x) else False


  # Main Newton-Raphson algorithm
  def newtonRaphson(self):
    X = [self.x] # Store the results in X
    local_variable_errors = [[self.x[i].item() for i in range(len(self.x))]]  # Store local variable erros
    local_errors = []   # Store local L^2 norm errors
    counter = 0
    while counter < self.maxIterations and self.islocalErrorGreaterThanTolerance(counter, local_variable_errors):

      # Update the counter
      counter += 1  

      # Calculate the jacobian and find a new solution given previous solution
      X.append(th.sub(self.x, th.transpose(th.mm(self.inverseJacobian(), th.transpose(self.functionCalculation(), 0, 1)), 0, 1)))

      # Calculate the local error for each unknown variables and store it in an array 
      # Calculate the local error for all unknown variables
      local_variable_errors.append([])
      local_error = 0
      for i in range(len(self.x)):
        local_variable_errors[counter].append(abs(X[counter][0][i].item() - self.x[i].item()))
        local_error += (X[counter][0][i].item() - self.x[i].item())**2

      local_errors.append(local_error**(1./2.))

      # Update the solution x
      self.x = th.tensor(list(map(lambda x: x.item(), list(X[counter][0]))), dtype=th.float64)

    return X, local_variable_errors, local_errors

  # Exact error
  def exact_error(self, exact_solution, approx_solution):
    exact_error = 0
    for i in range(len(self.x)):
      exact_error += (exact_solution[i] - approx_solution[-1][0][i].item())**2

    return exact_error**(1./2.)

  # Graph the error for all variables
  def graphError(self, error):
    fig = plt.figure(1)
    plt.title("Error")
    plt.xlabel("Iterations")
    plt.ylabel("Error")
    plt.plot(error)
    plt.grid()  
    plt.show()

In [ ]:
# Initial X
initialX = th.tensor([1.02, 0.5], dtype=th.float64)

# Functions
functions = []
functions.append(lambda x: x[0]+x[0]*x[1]-4)
functions.append(lambda x: x[0]+x[1]-3)
tolerance = 1e-08

# Real solutions
exact_solution = [2., 1.]

# Max iterations
maxIterations = 200

NR = NewtonRaphson(initialX, tolerance, functions, maxIterations)
x, variable_erros, local_errors = NR.newtonRaphson()

# Show X results
print('X results: \n')
print(x[0])
for solution in range(1,len(x)):
  for variable in range(len(initialX)):
    print(x[solution][0][variable].item(), end=" ")
  print()

# Show variable error results
print('\nVariable error results: \n')
for var_err in variable_erros:
  print(var_err)

# Show local L^2 norm error
print('\nL^2 local error results: \n')
for lcl_err in local_errors:
  print(lcl_err)

# Show the exact error
print(f'\nThe exact error between x* and x is: {NR.exact_error(exact_solution, x)}')

# Graph the error for one variable
NR.graphError(local_errors)